In [ ]:
# pre-processing method II
# 7 classes

In [46]:
import pandas as pd
import string
import re
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

import numpy as np

In [47]:
politifact = pd.read_json(r'./data/politifact.json')
snopes = pd.read_json(r'./data/snopes.json')

In [48]:
# column names
poli_cols = politifact.columns.to_list()
snop_cols = snopes.columns.to_list()

# poli_row_count, poli_col_count = politifact.shape
# snop_row_count, snop_col_count = snopes.shape

In [49]:
common_cols = set(poli_cols) & set(snop_cols)

td = pd.concat([snopes[list(common_cols)], politifact[list(common_cols)]])
td = td.reindex(columns = ['doc', 'claim', 'factchecker', 'url', 'sources', 'topic', 'published' ,'label' ])


In [50]:
mt = {
        '\n':" ",
        "'":None,
        '"':None,
        '“':None,
        '”':None,
        '’':None,
        '‘':None
    }
for n in range(10):
    mt[str(n)]=None

for x in string.punctuation:
    mt[x]=None
cols = td.columns.to_list()
cols.remove('label')

# cleaning the first file
for col in tqdm(cols):
    for indx, x in enumerate(td[col]):
        try:
            td.loc[indx,col] = (x.translate(x.maketrans(mt))).lower()
        except:
            if x==None:
                td.loc[indx,col] = ""
            elif type(x) == list:
                p = " ".join(str(z) for z in x)
                td.loc[indx,col] = p.translate(p.maketrans(mt)).lower()
            else:
                print(f"column:{col}\trow:{indx}\tvalue: {x} is invalid\tvalue type: {type(x)}")

100%|█████████████████████████████████████████████████████████████████████████████████████| 7/7 [03:07<00:00, 26.85s/it]


In [51]:
td = td.dropna(subset=['doc','claim','label'])

In [52]:
valid_labels = ['false','true','half-true','mostly-true','barely-true','pants-fire','mixture']

td['label']=td.label.map(lambda x: x.lower())
td = td[td.label.isin(valid_labels)]

In [53]:
td = td.reset_index(drop=True)

In [54]:
one_hot_prep = [one_hot(series['claim']+' '+ series['doc'],10000) for _, series in td.iterrows()]

In [55]:
temp = [len(x) for x in one_hot_prep]
print(max(temp))

6955


In [56]:
sent_len = 300
embedded_docs = pad_sequences(one_hot_prep,padding='pre',truncating='pre',maxlen=sent_len)

In [21]:
sent_len

300

In [57]:
# embedding_vector_fetures = 64
# vocab_size = 5000

# model = Sequential()
# model.add(Embedding(vocab_size,embedding_vector_fetures,input_length= sent_len))
# model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))#LSTM(100))
# model.add(Dense(24, activation = 'softmax'))
# model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
vocab_size=50000
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length= sent_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 100)          5000000   
                                                                 
 spatial_dropout1d_3 (Spatia  (None, 300, 100)         0         
 lDropout1D)                                                     
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                                 
 dense_3 (Dense)             (None, 7)                 707       
                                                                 
Total params: 5,081,107
Trainable params: 5,081,107
Non-trainable params: 0
_________________________________________________________________
None


In [63]:
X = np.array(embedded_docs)
Y = pd.get_dummies(td['label']).values

In [64]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(16699, 300) (4175, 300) (16699, 7) (4175, 7)


In [65]:
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test,y_test))

Epoch 1/10
522/522 [==============================] - 107s 202ms/step - loss: 1.8083 - accuracy: 0.2819 - val_loss: 1.8089 - val_accuracy: 0.2752
Epoch 2/10
522/522 [==============================] - 105s 202ms/step - loss: 1.7888 - accuracy: 0.2853 - val_loss: 1.8217 - val_accuracy: 0.2683
Epoch 3/10
522/522 [==============================] - 105s 201ms/step - loss: 1.7662 - accuracy: 0.2894 - val_loss: 1.8404 - val_accuracy: 0.2601
Epoch 4/10
522/522 [==============================] - 105s 201ms/step - loss: 1.7365 - accuracy: 0.3010 - val_loss: 1.8652 - val_accuracy: 0.2371
Epoch 5/10
522/522 [==============================] - 105s 201ms/step - loss: 1.6974 - accuracy: 0.3230 - val_loss: 1.8939 - val_accuracy: 0.2261
Epoch 6/10
522/522 [==============================] - 105s 201ms/step - loss: 1.6549 - accuracy: 0.3377 - val_loss: 1.9175 - val_accuracy: 0.2247
Epoch 7/10
522/522 [==============================] - 105s 202ms/step - loss: 1.6081 - accuracy: 0.3500 - val_loss: 1.9709 -

In [67]:
loss, accuracy = model.evaluate(x_test,y_test)
print(accuracy,loss)

131/131 [==============================] - 4s 31ms/step - loss: 2.1907 - accuracy: 0.2103
0.21029940247535706 2.190718650817871
